<a href="https://colab.research.google.com/github/BunnyNoBugs/coursework-semantic-roles/blob/main/rubert_rlc_error_detection_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

     |████████████████████████████████| 1.9MB 6.6MB/s 
     |████████████████████████████████| 3.2MB 32.2MB/s 
     |████████████████████████████████| 890kB 37.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=31e6a43af9555610bb9cdc63e86370b5a61b849167810d24ef28cd555030d0ca
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [34]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel, BertForSequenceClassification
from collections import Counter
import torch

In [3]:
rlc_test = pd.read_csv('https://raw.githubusercontent.com/BunnyNoBugs/coursework-semantic-roles/main/rlc_test.csv', index_col=[0], converters={'tags': eval})
rlc_test

,original_text,tags
45918,"Иногда я встаю утром, иду в университет, после...","[infl, other]"
33168,"Мы познакомились с людям ,работающим в Сорбонне.","[gov, syntax]"
41427,"Эму надо знать, что делать и какие последствия...",[ortho]
38020,"Ко мне всегда относются ка к 100%-ной финке, д...","[other, conj, infl, ortho, lex, space]"
6533,В начале телеведущий рассказывает немного ист...,"[conj, other]"
...,...,...
59817,Мы с ним за короткое время создали немало науч...,"[syntax, other]"
33144,За этими старыми зданиями вы увидете большую п...,"[ortho, infl, other]"
33638,В детсве я часто ходила к родственной бабушке.,"[lex, other]"
30157,Тут мне звонок и мамин взволнованный голос соо...,[lex]


In [116]:
rlc_tags = []
for i in range(len(rlc_test)):
    rlc_tags.extend(rlc_test.iloc[i]['tags'])
rlc_tags = Counter(rlc_tags)
rlc_tags = [(k, v) for k, v in rlc_tags.items()]
rlc_tags

[('infl', 405),
 ('other', 2982),
 ('gov', 776),
 ('syntax', 505),
 ('ortho', 1462),
 ('conj', 363),
 ('lex', 1542),
 ('space', 233),
 ('cs', 40),
 ('asp', 377),
 ('prep', 412),
 ('refl', 85),
 ('hyphen', 103),
 ('idiom', 117),
 ('ref', 322),
 ('misspell', 89),
 ('altern', 89),
 ('wo', 329),
 ('brev', 80),
 ('translit', 30),
 ('genneg', 5),
 ('agrgender', 236),
 ('num', 226),
 ('tense', 100),
 ('par', 74),
 ('argstr', 47),
 ('agrnum', 269),
 ('deriv', 191),
 ('aux', 75),
 ('mode', 35),
 ('neg', 32),
 ('graph', 49),
 ('agrpers', 26),
 ('passive', 23),
 ('agrgerund', 13),
 ('impers', 24),
 ('com', 8),
 ('gender', 59),
 ('argcase', 8)]

In [117]:
rlc_tags = pd.DataFrame(rlc_tags, columns=['tag', 'count'])
rlc_tags = rlc_tags.set_index('tag')
rlc_tags['frac'] = rlc_tags['count'] / rlc_tags['count'].sum()
rlc_tags = rlc_tags.sort_values(by=['count'], ascending=False)
rlc_tags

,count,frac
tag,,
other,2982,0.251837
lex,1542,0.130225
ortho,1462,0.123469
gov,776,0.065535
syntax,505,0.042648
prep,412,0.034794
infl,405,0.034203
asp,377,0.031839
conj,363,0.030656


In [46]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Курсовая по семантическим ролям/rubert_rlc_error_detection')
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

In [102]:
def perform_rlc_inference(text, model, tokenizer):
    text_tokenized = tokenizer(text, return_tensors='pt')
    results = torch.softmax(model(**text_tokenized).logits, dim=1).tolist()[0]

    return results

In [101]:
perform_rlc_inference('ыва', model, tokenizer)

[0.9995297193527222, 0.0004702805308625102]

In [87]:
rlc_tags.loc['infl']['responses'] 

0.0

In [121]:
%%time

rlc_tags['responses'] = 0
rlc_tags['prob_sum'] = 0.0

for i in range(len(rlc_test)):
    results = perform_rlc_inference(rlc_test.iloc[i]['original_text'], model, tokenizer)
    detected_class = round(results[1])
    incorrect_prob = results[1]

    for tag in rlc_test.iloc[i]['tags']:
        rlc_tags.at[tag, 'responses'] = rlc_tags.at[tag, 'responses'] + detected_class
        rlc_tags.at[tag, 'prob_sum'] = rlc_tags.at[tag, 'prob_sum'] + incorrect_prob

CPU times: user 11min 31s, sys: 4.36 s, total: 11min 36s
Wall time: 11min 38s


In [125]:
mean_inference_time = (60 * 11 + 31) / 5000
print(f'Mean inference time: {mean_inference_time} sec')

Mean inference time: 0.1382 sec


In [128]:
test_accuracy = rlc_tags['responses'].sum() / rlc_tags['count'].sum()
print(f'Accuracy on test: {test_accuracy}')

Accuracy on test: 0.664470906173465


In [129]:
rlc_tags['accuracy'] = rlc_tags['responses'] / rlc_tags['count']
rlc_tags['accuracy_non-strict'] = rlc_tags['prob_sum'] / rlc_tags['count']
rlc_tags

,count,frac,responses,prob_sum,accuracy,accuracy_non-strict
tag,,,,,,
other,2982,0.251837,1908,1891.886745,0.639839,0.634436
lex,1542,0.130225,928,931.809888,0.601816,0.604287
ortho,1462,0.123469,1177,1148.697730,0.805062,0.785703
gov,776,0.065535,527,519.013439,0.679124,0.668832
syntax,505,0.042648,333,329.044398,0.659406,0.651573
prep,412,0.034794,265,261.341713,0.643204,0.634325
infl,405,0.034203,303,294.830232,0.748148,0.727976
asp,377,0.031839,224,221.375839,0.594164,0.587204
conj,363,0.030656,215,212.025039,0.592287,0.584091


In [133]:
rlc_tags['accuracy'].corr(rlc_tags['count'])

-0.10252175121628129

In [131]:
rlc_tags['accuracy_non-strict'].corr(rlc_tags['count'])

-0.08573449450590478